Item-Based Collaborative Filtering Recommendations


In [53]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/zomato.csv')

# Data preprocessing
# Drop unnecessary columns
df.drop(['url', 'phone', 'dish_liked', 'menu_item'], axis=1, inplace=True)

# Remove duplicates
df.drop_duplicates(inplace=True)

# Remove rows with missing values
df.dropna(inplace=True)

# Convert 'rate' column to numeric, handle non-numeric values
def convert_rate_to_numeric(rate):
    if isinstance(rate, str) and '/' in rate:
        return float(rate.split('/')[0])
    else:
        return np.nan

df['rate'] = df['rate'].apply(convert_rate_to_numeric)

# Drop rows with non-numeric rate values
df.dropna(subset=['rate'], inplace=True)

# Pivot the dataframe to create item-user interaction matrix
item_user_matrix = df.pivot_table(index='location', columns='name', values='rate', fill_value=0)

# Compute item similarity matrix using cosine similarity
item_similarity_matrix = cosine_similarity(item_user_matrix.T)

# Recommendation function for item-based collaborative filtering
def recommend_item_based(name, top_n=10):
    global item_similarity_matrix, item_user_matrix

    # Check if the restaurant name exists in the DataFrame
    if name not in item_user_matrix.columns:
        print(f"Restaurant '{name}' not found.")
        return None

    # Find the index of the restaurant entered
    idx = item_user_matrix.columns.get_loc(name)

    # Find similar items and compute weighted average ratings
    similar_items = item_similarity_matrix[idx].argsort()[::-1][1:]  # Exclude the item itself
    weighted_ratings = item_user_matrix.values[:, similar_items] @ item_similarity_matrix[idx, similar_items]
    weighted_similarity = item_similarity_matrix[idx, similar_items].sum()

    # Compute final recommendation scores
    recommendation_scores = weighted_ratings / weighted_similarity

    # Sort recommendation scores and get top recommendations
    top_recommendations_idx = recommendation_scores.argsort()[::-1][:top_n]

    # Return the top recommended restaurants
    return item_user_matrix.columns[top_recommendations_idx]

# Example usage for item-based collaborative filtering
recommendations_item_based = recommend_item_based('Pai Vihar')
print("Item-Based Collaborative Filtering Recommendations:")
print(recommendations_item_based)


Item-Based Collaborative Filtering Recommendations:
Index(['A1 Biriyani Point', '1947', '29 Degrees North',
       '36th Cross Coffee Mane', '7 Hills Andhra Mess',
       '24 Carats - The Capitol', '1000 B.C', 'A1 Spice Inn',
       '24@43 - The Oterra', '@ The Bbq'],
      dtype='object', name='name')
